Test your network with different numbers of layers and neurons and present the best result found.

Describe the problem that you have chosen.

1. If possible, provide a link (kaggle?) with details on the problem/dataset.
2. What is the input? What is the output?
3. What type of problem is this?
4. What are the features or attributes that you will use from the dataset.
5. What is your evaluation metric? Accuracy? RMSE? MAE? Recall/Precision?

# Creating the dataset


In [13]:
%pip install yfinance
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install pandas_datareader
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt

import pandas_datareader as data_reader
import yfinance as yf
from pandas_datareader import data as pdr

yf.pdr_override()


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
df = pdr.get_data_yahoo("AMZN", start="2019-01-01", end="2020-01-01")

[*********************100%%**********************]  1 of 1 completed


In [15]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,73.260002,77.667999,73.046501,76.956497,76.956497,159662000
2019-01-03,76.000504,76.900002,74.855499,75.014000,75.014000,139512000
2019-01-04,76.500000,79.699997,75.915497,78.769501,78.769501,183652000
2019-01-07,80.115501,81.727997,79.459503,81.475502,81.475502,159864000
2019-01-08,83.234497,83.830498,80.830498,82.829002,82.829002,177628000
...,...,...,...,...,...,...
2019-12-24,89.690498,89.778503,89.378998,89.460503,89.460503,17626000
2019-12-26,90.050499,93.523003,89.974998,93.438499,93.438499,120108000
2019-12-27,94.146004,95.070000,93.300499,93.489998,93.489998,123732000


In [16]:
df = df.reset_index()

In [17]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-02,73.260002,77.667999,73.046501,76.956497,76.956497,159662000
1,2019-01-03,76.000504,76.900002,74.855499,75.014000,75.014000,139512000
2,2019-01-04,76.500000,79.699997,75.915497,78.769501,78.769501,183652000
3,2019-01-07,80.115501,81.727997,79.459503,81.475502,81.475502,159864000
4,2019-01-08,83.234497,83.830498,80.830498,82.829002,82.829002,177628000
...,...,...,...,...,...,...,...
247,2019-12-24,89.690498,89.778503,89.378998,89.460503,89.460503,17626000
248,2019-12-26,90.050499,93.523003,89.974998,93.438499,93.438499,120108000
249,2019-12-27,94.146004,95.070000,93.300499,93.489998,93.489998,123732000
250,2019-12-30,93.699997,94.199997,92.030998,92.344498,92.344498,73494000


In [ ]:
print("Hello Kato")